In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

from vllm import LLM, SamplingParams

/home/aicombined/miniconda3/envs/poc/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-06-17 16:30:52,316	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


- Instruction Formatting 전처리 함수

In [2]:
def preprocess_text(x):
    return f"###입력:{x}\n\n###출력:"

- merge_16bit로 저장된 모델 weight의 폴더를 이용하여 llm를 로드

In [3]:
llm = LLM(model="huggingface_merged_weights/SOLAR/", enable_lora=True)

INFO 06-17 16:30:59 llm_engine.py:100] Initializing an LLM engine (v0.4.2) with config: model='huggingface_merged_weights/SOLAR/', speculative_config=None, tokenizer='huggingface_merged_weights/SOLAR/', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=4096, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), seed=0, served_model_name=huggingface_merged_weights/SOLAR/)
INFO 06-17 16:30:59 utils.py:660] Found nccl from library /home/aicombined/.config/vllm/nccl/cu12/libnccl.so.2.18.1
INFO 06-17 16:31:00 selector.py:81] Cannot use FlashAttention-2 backend because the flash_attn package is not found. Please install it for better performance.
INFO 06-17 16:31:00 sele

- 자연어 생성을 위한 그리드 서치 디코딩 파라미터 선언

In [4]:
sampling_params = SamplingParams(max_tokens=256, temperature=0.5, top_k=50)

- 입력 데이터 전처리

In [5]:
system_prompt = """당신은 경영 컨설턴트 비서 인공지능입니다. 답이 어렵다면 단계별로 설명해주세요.\n\n"""
instruction = "ESG 경영이란 무엇인가?"
texts = [preprocess_text(system_prompt + instruction)]

- inference

In [6]:
outputs = llm.generate(texts, sampling_params)

Processed prompts: 100%|██████████| 1/1 [00:08<00:00,  8.89s/it]


- 생성 결과

In [7]:
for output in outputs:
    prompt = output.prompt
    generated_text = output.outputs[0].text
prompt, generated_text

('###입력:당신은 경영 컨설턴트 비서 인공지능입니다. 답이 어렵다면 단계별로 설명해주세요.\n\nESG 경영이란 무엇인가?\n\n###출력:',
 'ESG 경영은 기업이 사회적 책임, 환경 지속 가능성 및 윤리적 지배 구조에 대한 접근 방식을 강조하는 경영 방식입니다. 이는 기업이 사회적 책임을 이행하고 환경적 영향을 최소화하며 공정하고 투명한 지배 구조를 유지하는 것을 의미합니다. ESG 경영 접근 방식은 기업의 지속 가능성을 향상시키고 장기적으로 성과를 향상시키는 데 도움이 될 수 있습니다. 이는 기업의 재무적 건전성과 이익을 향상시키는 데 도움이 될 수 있습니다.$&%')